In [1]:
#%% 匯入功能
# 功能模組
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#TEJ API
import tejapi
tejapi.ApiConfig.api_key = "The Key"

In [2]:
#%% 匯入 TEJ資料
etf = tejapi.get('TWN/AEHOLD',
                 coid = '00881',
                 mdate= {'gte': '2021-09-01','lte':'2021-11-30'},
                 opts={'columns':['mdate', 'no','pct']},
                 chinese_column_name=True,paginate=True)

# 標竿指數：臺50指數
benchmark = tejapi.get('TWN/AIDXS',
                       coid = 'TWN50',
                       mdate= {'gte': '2021-09-01','lte':'2021-11-30'},
                       opts={'columns':['mdate','key3','mv_pct']},
                       chinese_column_name=True,paginate=True)


etf = etf[~etf['標的名稱'].isin(['申贖應付款','保證金','現金'])]
etf['證券碼'] = etf['標的名稱'].str[0:4]
etf['證券碼'] = np.where(etf['證券碼'] == 'TX 台','Y9999',etf['證券碼'])
etf['年'] = etf['日期'].dt.year
etf['月'] = etf['日期'].dt.month
etf = etf.drop_duplicates(subset=['年','月','證券碼'], keep='first')

benchmark['證券碼'] = benchmark['成份股'].str[0:4]
benchmark['年'] = benchmark['年月日'].dt.year
benchmark['月'] = benchmark['年月日'].dt.month
benchmark = benchmark.drop_duplicates(subset=['年','月','證券碼'], keep='first')

etf.head(10)

,日期,標的名稱,權重,證券碼,年,月
None,,,,,,
3,2021-09-01 00:00:00+00:00,2303 聯電,6.41,2303,2021,9
4,2021-09-01 00:00:00+00:00,2317 鴻海,11.99,2317,2021,9
5,2021-09-01 00:00:00+00:00,2324 仁寶,1.05,2324,2021,9
6,2021-09-01 00:00:00+00:00,2330 台積電,30.24,2330,2021,9
7,2021-09-01 00:00:00+00:00,2345 智邦,1.73,2345,2021,9
8,2021-09-01 00:00:00+00:00,2352 佳世達,0.55,2352,2021,9
9,2021-09-01 00:00:00+00:00,2356 英業達,0.84,2356,2021,9
10,2021-09-01 00:00:00+00:00,2360 致茂,0.80,2360,2021,9
11,2021-09-01 00:00:00+00:00,2379 瑞昱,3.05,2379,2021,9


In [3]:
# 獲得 etf與 benchmark的代碼
coid_list = etf['證券碼'].unique().tolist()
coid_list.append('Y9999')
coid_list = coid_list + benchmark['證券碼'].unique().tolist()

# 抓取公司的產業名稱
code = tejapi.get("TWN/EWNPRCSTD",
                  coid = coid_list,
                  paginate=True,
                  opts={'columns':['coid', 'coid_name','tseindnm']},
                  chinese_column_name=True)

code.head(5)

,證券碼,證券名稱,TSE產業名
None,,,
0,1101,台泥,水泥工業
1,1102,亞泥,水泥工業
2,1216,統一,食品工業
3,1301,台塑,塑膠工業
4,1303,南亞,塑膠工業


In [4]:
# 股價
price = tejapi.get('TWN/AAPRCM1',
                   coid = coid_list,
                   mdate= {'gte': '2021-09-01','lte':'2021-11-30'},
                   opts={'columns':['coid', 'mdate','roi']},
                   chinese_column_name=True,
                   paginate=True)

price['年'] = price['年月'].dt.year
price['月'] = price['年月'].dt.month

price.head(5)

,證券代碼,年月,報酬率％_月,年,月
None,,,,,
0,1101,2021-09-01 00:00:00+00:00,5.2631,2021,9
1,1101,2021-10-01 00:00:00+00:00,-5.3920,2021,10
2,1101,2021-11-01 00:00:00+00:00,-4.6630,2021,11
3,1102,2021-09-01 00:00:00+00:00,1.5589,2021,9
4,1102,2021-10-01 00:00:00+00:00,-2.8506,2021,10


In [5]:
#%% 合併產業名稱 ETF

# 合併產業名稱
etf = pd.merge(etf ,code , how = 'left' , on = ['證券碼'])
etf = pd.merge(etf ,price ,how = 'left' , left_on=['年','月','證券碼'], right_on=['年','月','證券代碼'])

benchmark = pd.merge(benchmark ,code , how = 'left' , on = ['證券碼'])
benchmark = pd.merge(benchmark ,price ,how = 'left' ,
                     left_on=['年','月','證券碼'], right_on=['年','月','證券代碼'])

# 處理產業不一致問題
# 若 benchmark的產業種類沒有在 etf的產業種類中找到，則 benchmark中特殊的產業改成其他
benchmark['TSE產業名'] = np.where(benchmark['TSE產業名'].isin(etf['TSE產業名'].unique().tolist()),
                               benchmark['TSE產業名'],'其他')

# 若 etf的產業種類沒有在 benchmark的產業種類中找到，則 etf中特殊的產業改成其他
etf['TSE產業名'] = np.where(etf['TSE產業名'].isin(benchmark['TSE產業名'].unique().tolist()),
                          etf['TSE產業名'],'其他')

In [6]:
#%% 計算產業與標竿指數的月報酬率,權重

etf = etf.sort_values(by=['年','月','TSE產業名','證券代碼']).reset_index(drop=True) # 排序年月日
etf['TSE產業名'] = np.where(etf['TSE產業名'].isna(),'其他' ,etf['TSE產業名'])
etf['權重'] = etf['權重'] * 0.01


etf['產業權重'] = etf.groupby(['TSE產業名','年','月'])['權重'].transform('sum')
etf['實際當月報酬率'] = etf['權重'] * etf['報酬率％_月']
etf['產業當月報酬率'] = etf.groupby(['TSE產業名','年','月'])['實際當月報酬率'].transform('sum') / etf['產業權重']

etf['實際產業當月報酬率'] = etf['產業當月報酬率'] * etf['產業權重']
etf['ETF 當月報酬率'] = etf.groupby(['年','月'])['實際當月報酬率'].transform('sum')
etf = etf[['年','月','TSE產業名','標的名稱','權重','報酬率％_月','產業權重','產業當月報酬率']]


benchmark = benchmark.sort_values(by=['年','月','TSE產業名','證券代碼']).reset_index(drop=True) # 排序年月日
benchmark = benchmark[['年月日','TSE產業名','成份股','證券代碼','年','月','前日市值比重','報酬率％_月']]

benchmark['前日市值比重'] = benchmark['前日市值比重'] * 0.01
benchmark['產業權重'] = benchmark.groupby(['TSE產業名','年','月'])['前日市值比重'].transform('sum')
benchmark['實際當月報酬率'] = benchmark['前日市值比重'] * benchmark['報酬率％_月']
benchmark['產業當月報酬率'] = benchmark.groupby(['TSE產業名','年','月'])['實際當月報酬率'].transform('sum') \
    / benchmark['產業權重']

benchmark['實際產業當月報酬率'] = benchmark['產業當月報酬率'] * benchmark['產業權重']
benchmark['ETF 當月報酬率'] = benchmark.groupby(['年','月'])['實際當月報酬率'].transform('sum')

benchmark.head(5)

,年月日,TSE產業名,成份股,證券代碼,年,月,前日市值比重,報酬率％_月,產業權重,實際當月報酬率,產業當月報酬率,實際產業當月報酬率,ETF 當月報酬率
0,2021-09-01 00:00:00+00:00,光電業,2409 友達,2409,2021,9,0.005084,0.2842,0.013939,0.001445,-11.511265,-0.160456,-3.615165
1,2021-09-01 00:00:00+00:00,光電業,3008 大立光,3008,2021,9,0.008855,-18.2835,0.013939,-0.161900,-11.511265,-0.160456,-3.615165
2,2021-09-01 00:00:00+00:00,其他,1101 台泥,1101,2021,9,0.008478,5.2631,0.274339,0.044621,-0.963335,-0.264280,-3.615165
3,2021-09-01 00:00:00+00:00,其他,1102 亞泥,1102,2021,9,0.003597,1.5589,0.274339,0.005607,-0.963335,-0.264280,-3.615165
4,2021-09-01 00:00:00+00:00,其他,1216 統一,1216,2021,9,0.011469,-1.7368,0.274339,-0.019919,-0.963335,-0.264280,-3.615165


In [7]:
#%% 合併 ETF與 Benchmark

benchmark = benchmark.drop_duplicates(subset=['TSE產業名','年','月'], keep='first').reset_index(drop=True)
benchmark = benchmark[['年','月','TSE產業名','產業權重','產業當月報酬率']].rename({'產業權重': '標竿權重',
                                                                   '產業當月報酬率':'標竿當月報酬率'}, axis=1)

etf = etf.drop_duplicates(subset=['TSE產業名','年','月'], keep='first').reset_index(drop=True)
etf = etf[['年','月','TSE產業名','產業權重','產業當月報酬率']].rename({'產業權重': '投組權重',
                                                               '產業當月報酬率':'投組當月報酬率'}, axis=1)

etf.head(5)

,年,月,TSE產業名,投組權重,投組當月報酬率
0,2021,9,光電業,0.0238,-18.283500
1,2021,9,其他,0.0272,-2.156355
2,2021,9,其他電子業,0.1279,-5.189256
3,2021,9,半導體,0.6297,-3.043494
4,2021,9,通信網路業,0.0553,-2.857103


In [8]:
#%% 績效歸因表

table = pd.merge(etf ,benchmark ,how = 'left' , on=['年','月','TSE產業名'])
table = table[table['月'] == 11]
table = table.drop(['年','月'], axis=1)
table = table.set_index(['TSE產業名'])
table = table.sort_values(by=['投組權重'], ascending=False)
table = table.fillna(0)


table['配置效果'] = (table['投組權重'] - table['標竿權重']) * \
    (table['標竿當月報酬率'] - sum(table[:7]['標竿權重'] * table[:7]['標竿當月報酬率']))

table['選擇效果'] = table['標竿權重'] * (table['投組當月報酬率'] - table['標竿當月報酬率'])
table['交互效果'] = (table['投組權重'] - table['標竿權重']) * (table['投組當月報酬率'] - table['標竿當月報酬率'])
table['主動報酬'] = table['配置效果'] + table['選擇效果'] + table['交互效果']

table.loc['合計',:] = table.sum(axis=0)
table.loc['合計','投組當月報酬率'] = sum(table[:7]['投組權重'] * table[:7]['投組當月報酬率'])
table.loc['合計','標竿當月報酬率'] = sum(table[:7]['標竿權重'] * table[:7]['標竿當月報酬率'])

table = table.round(2)
table

,投組權重,投組當月報酬率,標竿權重,標竿當月報酬率,配置效果,選擇效果,交互效果,主動報酬
TSE產業名,,,,,,,,
半導體,0.61,4.93,0.60,2.47,0.01,1.46,0.04,1.52
其他電子業,0.12,-2.91,0.04,-3.27,-0.39,0.02,0.03,-0.34
電子零組件,0.11,11.80,0.03,7.50,0.46,0.12,0.34,0.93
電腦及週邊,0.07,1.55,0.03,3.48,0.07,-0.05,-0.08,-0.06
通信網路業,0.05,5.01,0.02,0.65,-0.03,0.10,0.12,0.19
光電業,0.02,-3.86,0.01,-0.03,-0.02,-0.05,-0.03,-0.10
其他,0.02,0.32,0.27,0.30,0.36,0.00,-0.00,0.36
合計,1.00,4.22,1.00,1.73,0.47,1.61,0.41,2.49


In [ ]:
#%% 製作三個月雷達圖

fig = go.Figure()
date = 10
for date in etf['月'].unique():
    table = pd.merge(etf ,benchmark ,how = 'left' , on=['年','月','TSE產業名'])
    table = table[table['月'] == date]
    table = table.drop(['年','月'], axis=1)
    table = table.set_index(['TSE產業名'])
    table = table.sort_values(by=['投組權重'], ascending=False)
    table = table.fillna(0)
    
    table['配置效果'] = (table['投組權重'] - table['標竿權重']) * \
        (table['標竿當月報酬率'] - sum(table[:7]['標竿權重'] * table[:7]['標竿當月報酬率']))
        
    table['選擇效果'] = table['標竿權重'] * (table['投組當月報酬率'] - table['標竿當月報酬率'])
    table['交互效果'] = (table['投組權重'] - table['標竿權重']) * (table['投組當月報酬率'] - table['標竿當月報酬率'])
    table['主動報酬'] = table['配置效果'] + table['選擇效果'] + table['交互效果']
    
    table.loc['合計',:] = table.sum(axis=0)
    table.loc['合計','投組當月報酬率'] = sum(table[:7]['投組權重'] * table[:7]['投組當月報酬率'])
    table.loc['合計','標竿當月報酬率'] = sum(table[:7]['標竿權重'] * table[:7]['標竿當月報酬率'])
    
    table = table.round(2)
    table.index = ['半導體', '其他電子業', '電子零組件', '電腦及週邊', '通信網路業', '光電業','其他','合計']
    
    fig.add_trace(go.Scatterpolar(r= table.drop(['合計']).loc[:,'主動報酬'],
                                  theta= table.drop(['合計']).index,
                                  fill='toself',
                                  name=str(date) + '月'))

fig.show()